In [155]:
import pandas as pd 
from graphviz import Digraph
import numpy as np

In [156]:
df = pd.read_csv('mutations_1.csv')

In [157]:
# data = df
# data['class'] = data['class'].str.startswith('C').astype(int)
# data

### Calculate the total of samples and probability of Occurence

In [158]:
def Sample_sum(classes):
    # Assuming 'classes' is a numpy array containing the class labels
    classes = classes.astype(str)
    cancer_mask = np.char.startswith(classes, 'C')
    cancer_total = cancer_mask.sum()
    non_cancer_total = (~cancer_mask).sum()
    
    return cancer_total, non_cancer_total


In [159]:
def Probability(df):
    sample_total = len(df)
    cancer_total, non_cancer_total = Sample_sum(df['class'].values)
    if sample_total == 0:  # Handle empty DataFrame
        return 0, 0
    probability_C = cancer_total/sample_total
    probability_NC = non_cancer_total/sample_total
    
    return probability_C, probability_NC

In [160]:
total_df = len(df)
t_c, t_nc = Sample_sum(df['class'].values)
pC, pNC = Probability(df)
print(f"""For the root note, t \n
Total Number of Samples, n(t): {total_df} \n
Total Cancer Samples, n(t, C): {t_c} \n
Total NonCancer Samples, n(t, NC): {t_nc} \n
Probability of Selecting C Sample at Node, p(C|t): {pC} \n
Probability of Selecting NC Sample at Node, p(NC|t): {pNC}
""")

For the root note, t 

Total Number of Samples, n(t): 250 

Total Cancer Samples, n(t, C): 108 

Total NonCancer Samples, n(t, NC): 142 

Probability of Selecting C Sample at Node, p(C|t): 0.432 

Probability of Selecting NC Sample at Node, p(NC|t): 0.568



### Splitting and Computations

In [161]:
def Split_and_Compute(df):
    sample_total = len(df)
    classes = df['class'].values  # Convert to numpy array for faster processing
    results = []

    # Iterate over columns using numpy operations
    for column in df.columns[1:]:
        # Split data based on column feature into two groups
        feature_values = df[column].values
        left_mask = feature_values == 1
        right_mask = ~left_mask  # No need to compare to zero

        # Calculate total samples, cancer and non-cancer in each split
        left_cancer_tot, left_non_cancer_tot = Sample_sum(classes[left_mask])
        right_cancer_tot, right_non_cancer_tot = Sample_sum(classes[right_mask])
        
        # Calculate Proportions
        left_sum = left_mask.sum()
        right_sum = right_mask.sum()
        PL = left_sum / sample_total
        PR = right_sum / sample_total
        
        # Calculate Probabilities with numpy division that handles division by zero
        Prob_C_left, Prob_NC_left = np.divide([left_cancer_tot, left_non_cancer_tot], left_sum, out=np.zeros(2), where=left_sum != 0)

        Prob_C_right, Prob_NC_right = np.divide([right_cancer_tot, right_non_cancer_tot], right_sum, out=np.zeros(2), where=right_sum != 0)

        
        Q = abs(Prob_C_left - Prob_C_right) + abs(Prob_NC_left - Prob_NC_right)
        
        # Append results for this feature to the list
        results.append([left_sum, right_sum, left_cancer_tot, left_non_cancer_tot, PL, PR, Prob_C_left, 
                        Prob_NC_left, Prob_C_right, Prob_NC_right, 2*PL*PR, Q, 2*PL*PR*Q])
    
    # Convert results list to DataFrame
    result_df = pd.DataFrame(results, columns=['n(tL)', 'n(tR)', 'n(tL,C)', 'n(tL,NC)', 'PL', 'PR', 
                                               'P(C|tL)', 'P(NC|tL)', 'P(C|tR)', 'P(NC|tR)', '2PLPR', 'Q', 'Phi(s|t)'],
                             index=df.columns[1:])
    feature_table = result_df.sort_values(by='Phi(s|t)', ascending=False)
    
    return feature_table


In [162]:
table_1 = Split_and_Compute(df)

In [163]:
table_1.head(10)

,n(tL),n(tR),"n(tL,C)","n(tL,NC)",PL,PR,P(C|tL),P(NC|tL),P(C|tR),P(NC|tR),2PLPR,Q,Phi(s|t)
RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--,28,222,27,1,0.112,0.888,0.964286,0.035714,0.364865,0.635135,0.198912,1.198842,0.238464
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,23,227,22,1,0.092,0.908,0.956522,0.043478,0.378855,0.621145,0.167072,1.155334,0.193024
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,23,227,22,1,0.092,0.908,0.956522,0.043478,0.378855,0.621145,0.167072,1.155334,0.193024
PPP2R1A_GRCh37_19:52715971-52715971_Missense-Mutation_SNP_C-C-G_C-C-T,24,226,0,24,0.096,0.904,0.000000,1.000000,0.477876,0.522124,0.173568,0.955752,0.165888
KRAS_GRCh37_12:25398284-25398284_Missense-Mutation_SNP_C-C-A_C-C-T_C-C-G,28,222,22,6,0.112,0.888,0.785714,0.214286,0.387387,0.612613,0.198912,0.796654,0.158464
NCOA3_GRCh37_20:46282985-46282985_3'Flank_DEL_T-T--,15,235,15,0,0.060,0.940,1.000000,0.000000,0.395745,0.604255,0.112800,1.208511,0.136320
DRD5_GRCh37_4:9785349-9785349_3'UTR_SNP_G-G-C,14,236,14,0,0.056,0.944,1.000000,0.000000,0.398305,0.601695,0.105728,1.203390,0.127232
CTNNA2_GRCh37_2:80875615-80875615_3'UTR_DEL_T-T--,13,237,13,0,0.052,0.948,1.000000,0.000000,0.400844,0.599156,0.098592,1.198312,0.118144
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,13,237,13,0,0.052,0.948,1.000000,0.000000,0.400844,0.599156,0.098592,1.198312,0.118144
CSNK1G1_GRCh37_15:64461260-64461260_3'UTR_DEL_A-A--,13,237,13,0,0.052,0.948,1.000000,0.000000,0.400844,0.599156,0.098592,1.198312,0.118144


In [164]:
root_node = table_1.index[0]
root_node

'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--'

### find the left and right child of the root node based on F(s,t)

#### samples in left and right child, and their respective feature tables ordered by top F(s,t)

In [165]:
left_child = df[df[root_node]==1]
right_child = df[df[root_node]==0]

In [166]:
left_child

,class,GOT1_GRCh37_10:101163586-101163586_Missense-Mutation_SNP_C-C-T,TEX36_GRCh37_10:127371546-127371546_Nonsense-Mutation_SNP_G-G-A,KIAA1217_GRCh37_10:24810824-24810824_Missense-Mutation_SNP_C-C-T,SIRT1_GRCh37_10:69676051-69676051_Missense-Mutation_SNP_C-C-T,MYPN_GRCh37_10:69881845-69881845_Missense-Mutation_SNP_C-C-T,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,NPAT_GRCh37_11:108032094-108032094_Missense-Mutation_SNP_G-G-T_G-G-A,ARNTL_GRCh37_11:13402723-13402723_Silent_SNP_G-G-A,COPB1_GRCh37_11:14480092-14480092_Missense-Mutation_SNP_G-G-A_G-G-T,...,FBXO21_GRCh37_12:117624320-117624320_Frame-Shift-Del_DEL_A-A--,CLRN1_GRCh37_3:150644691-150644691_3'UTR_DEL_A-A--,VPS37A_GRCh37_8:17153149-17153149_3'UTR_DEL_A-A--,VANGL2_GRCh37_1:160398257-160398258_3'UTR_INS_----G,SRGAP3_GRCh37_3:9023972-9023972_3'UTR_DEL_A-A--,PTENP1_GRCh37_9:33674774-33674774_RNA_DEL_T-T--,KMT2C_GRCh37_7:151836804-151836804_Nonsense-Mutation_SNP_G-G-A,CDKN1C_GRCh37_11:2904846-2904846_3'UTR_DEL_T-T--,PIK3R2_GRCh37_19:18278070-18278070_Missense-Mutation_SNP_A-A-G,LMO7_GRCh37_13:76432354-76432354_3'UTR_SNP_C-C-T
8,C3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,C5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32,C11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35,C13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,C15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,C18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,C21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,C24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,C26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,C27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
left_feature_table = Split_and_Compute(left_child)
left_feature_table

,n(tL),n(tR),"n(tL,C)","n(tL,NC)",PL,PR,P(C|tL),P(NC|tL),P(C|tR),P(NC|tR),2PLPR,Q,Phi(s|t)
HGSNAT_GRCh37_8:43002099-43002099_Frame-Shift-Del_DEL_A-A--,1,27,0,1,0.035714,0.964286,0.0,1.0,1.000000,0.000000,0.068878,2.0,0.137755
CAPS2_GRCh37_12:75669988-75669988_3'UTR_DEL_T-T--,1,27,0,1,0.035714,0.964286,0.0,1.0,1.000000,0.000000,0.068878,2.0,0.137755
FAT3_GRCh37_11:92616097-92616097_Missense-Mutation_SNP_G-G-A,1,27,0,1,0.035714,0.964286,0.0,1.0,1.000000,0.000000,0.068878,2.0,0.137755
TNS3_GRCh37_7:47474967-47474967_Silent_SNP_C-C-T,1,27,0,1,0.035714,0.964286,0.0,1.0,1.000000,0.000000,0.068878,2.0,0.137755
TTC14_GRCh37_3:180327408-180327408_Intron_DEL_T-T--,1,27,0,1,0.035714,0.964286,0.0,1.0,1.000000,0.000000,0.068878,2.0,0.137755
...,...,...,...,...,...,...,...,...,...,...,...,...,...
MIR1283-1_GRCh37_19:54191812-54191812_RNA_SNP_G-G-A,0,28,0,0,0.000000,1.000000,0.0,0.0,0.964286,0.035714,0.000000,1.0,0.000000
FPR3_GRCh37_19:52327411-52327411_Missense-Mutation_SNP_G-G-A,0,28,0,0,0.000000,1.000000,0.0,0.0,0.964286,0.035714,0.000000,1.0,0.000000
NAPSA_GRCh37_19:50865026-50865026_Missense-Mutation_SNP_C-C-T,0,28,0,0,0.000000,1.000000,0.0,0.0,0.964286,0.035714,0.000000,1.0,0.000000
MYH14_GRCh37_19:50758526-50758526_Silent_SNP_G-G-A,0,28,0,0,0.000000,1.000000,0.0,0.0,0.964286,0.035714,0.000000,1.0,0.000000


In [168]:
right_feature_table = Split_and_Compute(right_child)
right_feature_table

,n(tL),n(tR),"n(tL,C)","n(tL,NC)",PL,PR,P(C|tL),P(NC|tL),P(C|tR),P(NC|tR),2PLPR,Q,Phi(s|t)
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,14,208,14,0,0.063063,0.936937,1.000000,0.000000,0.322115,0.677885,0.118172,1.355769,0.160214
PPP2R1A_GRCh37_19:52715971-52715971_Missense-Mutation_SNP_C-C-G_C-C-T,24,198,0,24,0.108108,0.891892,0.000000,1.000000,0.409091,0.590909,0.192841,0.818182,0.157779
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,15,207,14,1,0.067568,0.932432,0.933333,0.066667,0.323671,0.676329,0.126004,1.219324,0.153640
KRAS_GRCh37_12:25398284-25398284_Missense-Mutation_SNP_C-C-A_C-C-T_C-C-G,22,200,16,6,0.099099,0.900901,0.727273,0.272727,0.325000,0.675000,0.178557,0.804545,0.143657
PTEN_GRCh37_10:89692904-89692904_Missense-Mutation_SNP_C-C-G,13,209,11,2,0.058559,0.941441,0.846154,0.153846,0.334928,0.665072,0.110259,1.022451,0.112734
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SDK2_GRCh37_17:71335062-71335062_Silent_SNP_G-G-A,0,222,0,0,0.000000,1.000000,0.000000,0.000000,0.364865,0.635135,0.000000,1.000000,0.000000
DNAH2_GRCh37_17:7696521-7696521_Nonsense-Mutation_SNP_C-C-T,0,222,0,0,0.000000,1.000000,0.000000,0.000000,0.364865,0.635135,0.000000,1.000000,0.000000
DHRS7C_GRCh37_17:9674865-9674865_Missense-Mutation_SNP_G-G-T,0,222,0,0,0.000000,1.000000,0.000000,0.000000,0.364865,0.635135,0.000000,1.000000,0.000000
CEP76_GRCh37_18:12701088-12701088_Nonsense-Mutation_SNP_C-C-A,0,222,0,0,0.000000,1.000000,0.000000,0.000000,0.364865,0.635135,0.000000,1.000000,0.000000


In [169]:
# left_node,right_node = left_feature_table.index[0], right_feature_table.index[0]
# print(f"The left node of the root is {left_node} \nThe right node of the root is {right_node}")

##### Leaf nodes of decision trees, split by node of the left and right of the root

In [170]:
# leaf_A1 = left_child[left_child[left_node]==1]
# leaf_A2 = left_child[left_child[left_node]==0]
# leaf_B1= right_child[right_child[right_node]==1]
# leaf_B2 = right_child[right_child[right_node]==0]

#### Function to get root node, left, right node

In [171]:
# get tree nodes
def get_tree(df):
    root_node = Split_and_Compute(df).index[0]
    left_child = df[df[root_node]==1]
    right_child = df[df[root_node]==0]
    left_node = Split_and_Compute(left_child).index[0]
    right_node = Split_and_Compute(right_child).index[0]
    return root_node, left_child, right_child, left_node, right_node

#### Lets get the classification conditions now - boolean values

In [172]:
def get_conditions(df):
    root_node, left_child, right_child, left_node, right_node = get_tree(df)
    print(f"The root node of the root is {root_node} \nThe left node of the root is {left_node} \nThe right node of the root is {right_node}")
    leaf_A1 = left_child[left_child[left_node]==1]
    leaf_A2 = left_child[left_child[left_node]==0]
    leaf_B1= right_child[right_child[right_node]==1]
    leaf_B2 = right_child[right_child[right_node]==0]
    
    # conditions
    A1 = leaf_A1['class'].str.startswith('C').astype(int).sum() > leaf_A1['class'].str.startswith('NC').astype(int).sum()
    A2 = leaf_A2['class'].str.startswith('C').astype(int).sum() > leaf_A2['class'].str.startswith('NC').astype(int).sum()
    B1 = leaf_B1['class'].str.startswith('C').astype(int).sum() > leaf_B1['class'].str.startswith('NC').astype(int).sum()
    B2 = leaf_B2['class'].str.startswith('C').astype(int).sum() > leaf_B2['class'].str.startswith('NC').astype(int).sum()
    
    conditions = [A1, A2, B1, B2]
    
    return conditions


### Draw the Decision Tree

In [173]:
# dot = Digraph()

# dot.node('A', root_node[:5])
# dot.node('B', left_node[:5])
# dot.node('C', right_node[:5])
# dot.node('D', 'A1')
# dot.node('E', 'A2')
# dot.node('F', 'B1')
# dot.node('G', 'B2')

# dot.edge('A', 'B')
# dot.edge('A', 'C')
# dot.edge('B', 'D')
# dot.edge('B', 'E')
# dot.edge('C', 'F')
# dot.edge('C', 'G')

# dot.view()

### Classification

In [174]:
conditions = get_conditions(df)

The root node of the root is RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T-- 
The left node of the root is HGSNAT_GRCh37_8:43002099-43002099_Frame-Shift-Del_DEL_A-A-- 
The right node of the root is DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--


In [175]:
def Classification(df, sample, tree_nodes, conditions):
    S = df[df['class']==sample]
    root_node, left_node, right_node = tree_nodes
    A1, A2, B1, B2 = conditions
    if S[root_node].iloc[0] == 1:
        if S[left_node].iloc[0]==1:
            if A1: #false
                return "C Sample"
            else:
                return "NC Sample"
        else:
            if A2: #true
                return "C Sample"
            else:
                return "NC Sample"
    else:
        if S[right_node].iloc[0]==1:
            if B1: #true
                return "C Sample"
            else:
                return "NC Sample"
        else:
            if B2: #false
                return "C Sample"
            else:
                return "NC Sample"
        

#### Test Classification Logic on entire data frame here 

In [176]:
# tree_nodes = get_tree()
# Classification(df, 'NC49', tree_nodes, conditions)

In [177]:
# def Classify_test(df, test):
#     # get_tree_nodes
#     root_node = Split_and_Compute(df).index[0]
#     left_child = df[df[root_node]==1]
#     right_child = df[df[root_node]==0]
#     left_node = Split_and_Compute(left_child).index[0]
#     right_node = Split_and_Compute(right_child).index[0]
    
#     # tree_nodes = [root_node, left_node, right_node]
#     root_node, left_node, right_node
    
#     # get conditions
#     leaf_A1 = left_child[left_child[left_node]==1]
#     leaf_A2 = left_child[left_child[left_node]==0]
#     leaf_B1= right_child[right_child[right_node]==1]
#     leaf_B2 = right_child[right_child[right_node]==0]
    
#     # conditions
#     A1 = leaf_A1['class'].str.startswith('C').astype(int).sum() > leaf_A1['class'].str.startswith('NC').astype(int).sum()
#     A2 = leaf_A2['class'].str.startswith('C').astype(int).sum() > leaf_A2['class'].str.startswith('NC').astype(int).sum()
#     B1 = leaf_B1['class'].str.startswith('C').astype(int).sum() > leaf_B1['class'].str.startswith('NC').astype(int).sum()
#     B2 = leaf_B2['class'].str.startswith('C').astype(int).sum() > leaf_B2['class'].str.startswith('NC').astype(int).sum()
    
#     # conditions = [A1, A2, B1, B2]

#     root_mask = df[root_node] == 1
#     left_mask = df[left_node] == 1
#     right_mask = df[right_node] == 1
    
#     data = test.copy()
#     data['predicted_class'] = 'NC Sample'  # Default Value
#     data.loc[root_mask & left_mask, 'predicted_class'] = 'C Sample' if A1 else 'NC Sample'
#     data.loc[root_mask & ~left_mask, 'predicted_class'] = 'C Sample' if A2 else 'NC Sample'
#     data.loc[~root_mask & right_mask, 'predicted_class'] = 'C Sample' if B1 else 'NC Sample'
#     data.loc[~root_mask & ~right_mask, 'predicted_class'] = 'C Sample' if B2 else 'NC Sample'
    
#     classified_map = dict(zip(data['class'], data['predicted_class']))
    
#     return classified_map
    
    # my_map = {}
    # for i in range(len(test)):
    #     check = test.iloc[i]['class']
    #     my_map[check] = Classification(test, test.iloc[i], tree_nodes, conditions)
   
    # return my_map
    


### 3-fold Cross Validation

#### Build tree & Perform Classification

In [193]:
def build_tree_and_conditions(df):
    root_node = Split_and_Compute(df).index[0]
    left_child = df[df[root_node]==1]
    right_child = df[df[root_node]==0]
    left_node = Split_and_Compute(left_child).index[0]
    right_node = Split_and_Compute(right_child).index[0]
    
    leaf_A1 = left_child[left_child[left_node]==1]
    leaf_A2 = left_child[left_child[left_node]==0]
    leaf_B1= right_child[right_child[right_node]==1]
    leaf_B2 = right_child[right_child[right_node]==0]
    
    # A1 = leaf_A1['class'].str.startswith('C').astype(int).sum() > leaf_A1['class'].str.startswith('NC').astype(int).sum()
    # A2 = leaf_A2['class'].str.startswith('C').astype(int).sum() > leaf_A2['class'].str.startswith('NC').astype(int).sum()
    # B1 = leaf_B1['class'].str.startswith('C').astype(int).sum() > leaf_B1['class'].str.startswith('NC').astype(int).sum()
    # B2 = leaf_B2['class'].str.startswith('C').astype(int).sum() > leaf_B2['class'].str.startswith('NC').astype(int).sum()
    class_array = leaf_A1['class'].values.astype(str)
    A1 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    class_array = leaf_A2['class'].values.astype(str)
    A2 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    class_array = leaf_B1['class'].values.astype(str)
    B1 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    class_array = leaf_B2['class'].values.astype(str)
    B2 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    return root_node, left_node, right_node, A1, A2, B1, B2
    

def Classify_test(test, root_node, left_node, right_node, A1, A2, B1, B2):
    root_mask = test[root_node] == 1
    left_mask = test[left_node] == 1
    right_mask = test[right_node] == 1
    
    # data = test.copy()
    # data['predicted_class'] = 'NC Sample'
    # data.loc[root_mask & left_mask, 'predicted_class'] = 'C Sample' if A1 else 'NC Sample'
    # data.loc[root_mask & ~left_mask, 'predicted_class'] = 'C Sample' if A2 else 'NC Sample'
    # data.loc[~root_mask & right_mask, 'predicted_class'] = 'C Sample' if B1 else 'NC Sample'
    # data.loc[~root_mask & ~right_mask, 'predicted_class'] = 'C Sample' if B2 else 'NC Sample'
    predicted_classes = np.full(test.shape[0], 'NC Sample')
    predicted_classes[root_mask & left_mask] = 'C Sample' if A1 else 'NC Sample'
    predicted_classes[root_mask & ~left_mask] = 'C Sample' if A2 else 'NC Sample'
    predicted_classes[~root_mask & right_mask] = 'C Sample' if B1 else 'NC Sample'
    predicted_classes[~root_mask & ~right_mask] = 'C Sample' if B2 else 'NC Sample'
    
    classified_map = dict(zip(test['class'], predicted_classes))
    # classified_map = dict(zip(data['class'], data['predicted_class']))
    
    return classified_map

### Create confusion matrix and calculate classification metrics

In [247]:
def predicted_actual(test_map):
    test_df = pd.DataFrame(test_map, index=[0])
    test_df = test_df.transpose()
    test_df = test_df.reset_index()
    test_df.rename(columns={'index': 'ActualClass'}, inplace=True)
    test_df.rename(columns={0: 'PredictedClass'}, inplace=True)
    test_df['ActualClass'] = test_df['ActualClass'].str.startswith('C').astype(int)
    test_df['PredictedClass'] = test_df['PredictedClass'].str.startswith('C').astype(int)
    return test_df

def ConfusionMatrix(pred_actual_bool:pd.DataFrame):
    TP, TN, FN, FP = 0, 0, 0, 0
    for actual, predicted in zip(pred_actual_bool['ActualClass'], pred_actual_bool['PredictedClass']):
        if actual == predicted:
            if actual == 1:
                TP += 1
            else:
                TN += 1
        else: 
            if actual == 1:
                FN += 1
            else:
                FP += 1 
            
        values = [TP, TN, FP, FN]
    return values

# def ClassificationMetrics(conf_mat):
#     TP, TN, FP, FN = conf_mat
#     metrics = {}
#     metrics['Accuracy'] = round(((TP + TN) / (TP + TN + FP + FN)) * 100, 3)
#     metrics['Sensitivity'] = round(((TP)/(TP + FN))*100, 3)
#     metrics['Specificity'] = round(((TN)/(TN + FP))*100, 3)
#     metrics['Precision'] = round(((TP)/(TP + FP))*100, 3)
#     metrics['Miss_rate'] = round(((FN)/(TP + FN))*100, 3)
#     metrics['False_Discovery_Rate'] = round(((FP)/(TP + FP))*100, 3)
#     metrics['False_Omission_Rate'] = round(((FN)/(TN + FN))*100, 3)
    
#     return metrics
def ClassificationMetrics(conf_mat):
    TP, TN, FP, FN = conf_mat
    metrics = {}

    total = TP + TN + FP + FN
    metrics['Accuracy'] = round(((TP + TN) / total) * 100, 3) if total != 0 else 0

    metrics['Sensitivity'] = round((TP / (TP + FN)) * 100, 3) if (TP + FN) != 0 else 0
    metrics['Specificity'] = round((TN / (TN + FP)) * 100, 3) if (TN + FP) != 0 else 0
    metrics['Precision'] = round((TP / (TP + FP)) * 100, 3) if (TP + FP) != 0 else 0

    metrics['Miss_rate'] = round((FN / (TP + FN)) * 100, 3) if (TP + FN) != 0 else 0
    metrics['False_Discovery_Rate'] = round((FP / (TP + FP)) * 100, 3) if (TP + FP) != 0 else 0
    metrics['False_Omission_Rate'] = round((FN / (TN + FN)) * 100, 3) if (TN + FN) != 0 else 0
    
    return metrics


### training and testing function

In [180]:
def train_and_test(train, test, group):
    print(f"Starting Training and Testing for Group{group} \n")
    root_node, left_node, right_node, A1, A2, B1, B2 = build_tree_and_conditions(train)
    # print(f"{A1},{A2},{B1},{B2}")
    print(f"Decision Tree: {root_node[:5]}, {left_node[:5]}, {right_node[:5]}")
    classification_results = Classify_test(test, root_node, left_node, right_node, A1, A2, B1, B2)
    pred_actual_df = predicted_actual(classification_results)
    Confusion_Matrix_values = ConfusionMatrix(pred_actual_df)
    print("Confusion_Matrix:", Confusion_Matrix_values, '\n')
    metrics_final = ClassificationMetrics(Confusion_Matrix_values)
    print(metrics_final)
    return metrics_final

In [181]:
def split_data(df):
    split_1 = df.sample(frac=0.33, random_state=0)
    rem_data = df.drop(split_1.index)
    split_2 = rem_data.sample(frac=0.5, random_state=1)
    split_3 = rem_data.drop(split_2.index)
    
    return split_1, split_2, split_3 

In [182]:
split_1, split_2, split_3  = split_data(df)

In [194]:
train_A, test_A = pd.concat([split_1, split_2], axis=0), split_3
metrics_A = train_and_test(train_A, test_A, 'A')

Starting Training and Testing for GroupA 

Decision Tree: RPL22, GOT1_, PPP2R
Confusion_Matrix: [12, 44, 1, 27] 

{'Accuracy': 66.667, 'Sensitivity': 30.769, 'Specificity': 97.778, 'Precision': 92.308, 'Miss_rate': 69.231, 'False_Discovery_Rate': 7.692, 'False_Omission_Rate': 38.028}


In [195]:
train_B, test_B = pd.concat([split_2, split_3], axis=0), split_1
metrics_B = train_and_test(train_B, test_B, 'B')

Starting Training and Testing for GroupB 

Decision Tree: RPL22, PTEN_, KRAS_
Confusion_Matrix: [10, 46, 2, 24] 

{'Accuracy': 68.293, 'Sensitivity': 29.412, 'Specificity': 95.833, 'Precision': 83.333, 'Miss_rate': 70.588, 'False_Discovery_Rate': 16.667, 'False_Omission_Rate': 34.286}


In [196]:
train_C, test_C = pd.concat([split_1, split_3], axis=0), split_2
metrics_C = train_and_test(train_C, test_C, 'C')

Starting Training and Testing for GroupC 

Decision Tree: RPL22, ARHGA, RNF43
Confusion_Matrix: [7, 48, 1, 28] 

{'Accuracy': 65.476, 'Sensitivity': 20.0, 'Specificity': 97.959, 'Precision': 87.5, 'Miss_rate': 80.0, 'False_Discovery_Rate': 12.5, 'False_Omission_Rate': 36.842}


In [186]:
avg_met = {}
for key in metrics_A:
    avg_met[key] = str(((metrics_A[key] + metrics_B[key] + metrics_C[key])/3)) + '%'
    
avg_met

{'Accuracy': '66.812%',
 'Sensitivity': '26.727%',
 'Specificity': '97.19%',
 'Precision': '87.71366666666667%',
 'Miss_rate': '73.273%',
 'False_Discovery_Rate': '12.286333333333333%',
 'False_Omission_Rate': '36.38533333333333%'}

## Information Gain and Entropy 

In [205]:
def entropy(pC, pNC):
    # Ensure probabilities are non-zero before taking the log; if zero, the contribution to entropy is zero
    H_t = 0
    if pC > 0:
        H_t -= pC * np.log2(pC)
    if pNC > 0:
        H_t -= pNC * np.log2(pNC)
    return H_t

In [207]:
# Entropy
entropy_val = entropy(pC, pNC)
print(f"entropy of the dataset is: {entropy_val}")

entropy of the dataset is: 0.9866165198488093


In [249]:
def gain(df):
    sample_total = len(df)
    classes = df['class'].values  # Convert to numpy array for faster processing
    results = []
    
    # total_df_len = len(df)
    # total_c, total_nc = Sample_sum(df['class'].values)
    prob_C, prob_NC = Probability(df)
    
    # entropy calcaulation
    H_t = 0
    if prob_C > 0:
        H_t -= prob_C * np.log2(prob_C)
    if prob_NC > 0:
        H_t -= prob_NC * np.log2(prob_NC)

    # Iterate over columns using numpy operations
    for column in df.columns[1:]:
        # Split data based on column feature into two groups
        feature_values = df[column].values
        left_mask = feature_values == 1
        right_mask = ~left_mask  # No need to compare to zero

        # Calculate total samples, cancer and non-cancer in each split
        left_cancer_tot, left_non_cancer_tot = Sample_sum(classes[left_mask])
        right_cancer_tot, right_non_cancer_tot = Sample_sum(classes[right_mask])
        
        # Calculate Proportions
        left_sum = left_mask.sum()
        right_sum = right_mask.sum()
        PL = left_sum / sample_total
        PR = right_sum / sample_total
        
        # Calculate Probabilities with numpy division that handles division by zero
        Prob_C_left, Prob_NC_left = np.divide([left_cancer_tot, left_non_cancer_tot], left_sum, out=np.zeros(2), where=left_sum != 0)
        Prob_C_right, Prob_NC_right = np.divide([right_cancer_tot, right_non_cancer_tot], right_sum, out=np.zeros(2), where=right_sum != 0)

        # Entropy Calculation 
        H_tL, H_tR = 0, 0 
        if Prob_C_left > 0:
            H_tL -= Prob_C_left * np.log2(Prob_C_left)
        if Prob_NC_left > 0:
            H_tL -= Prob_NC_left * np.log2(Prob_NC_left)
            
        if Prob_C_right > 0:
            H_tR -= Prob_C_right * np.log2(Prob_C_right)
        if Prob_NC_right > 0:
            H_tR -= Prob_NC_right * np.log2(Prob_NC_right)
        
        H_st = (PL*H_tL) + (PR*H_tR)
        gain_s = H_t - H_st
        
        # Append results for this feature to the list
        results.append([left_sum, right_sum, left_cancer_tot, left_non_cancer_tot, right_cancer_tot, right_non_cancer_tot, PL, PR, H_st, H_t, gain_s])
    
    # Convert results list to DataFrame
    result_df = pd.DataFrame(results, columns=['n(tL)', 'n(tR)', 'n(tL,C)', 'n(tL,NC)', 'n(tR,C)', 'n(tR,NC)', 'PL', 'PR', 'H(s,t)', 'H(t)', 'gain(s)'],
                             index=df.columns[1:])
    feature_table = result_df.sort_values(by='gain(s)', ascending=False)
    
    return feature_table

In [250]:
table_gains = gain(df)

In [251]:
table_gains.head(10)

,n(tL),n(tR),"n(tL,C)","n(tL,NC)","n(tR,C)","n(tR,NC)",PL,PR,"H(s,t)",H(t),gain(s)
RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--,28,222,27,1,81,141,0.112,0.888,0.865519,0.986617,0.121098
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,23,227,22,1,86,141,0.092,0.908,0.892902,0.986617,0.093715
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,23,227,22,1,86,141,0.092,0.908,0.892902,0.986617,0.093715
PPP2R1A_GRCh37_19:52715971-52715971_Missense-Mutation_SNP_C-C-G_C-C-T,24,226,0,24,108,118,0.096,0.904,0.902723,0.986617,0.083894
NCOA3_GRCh37_20:46282985-46282985_3'Flank_DEL_T-T--,15,235,15,0,93,142,0.060,0.940,0.910302,0.986617,0.076314
DRD5_GRCh37_4:9785349-9785349_3'UTR_SNP_G-G-C,14,236,14,0,94,142,0.056,0.944,0.915633,0.986617,0.070983
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,13,237,13,0,95,142,0.052,0.948,0.920927,0.986617,0.065689
CSNK1G1_GRCh37_15:64461260-64461260_3'UTR_DEL_A-A--,13,237,13,0,95,142,0.052,0.948,0.920927,0.986617,0.065689
CTNNA2_GRCh37_2:80875615-80875615_3'UTR_DEL_T-T--,13,237,13,0,95,142,0.052,0.948,0.920927,0.986617,0.065689
ESRP1_GRCh37_8:95686611-95686611_Frame-Shift-Del_DEL_A-A--,12,238,12,0,96,142,0.048,0.952,0.926185,0.986617,0.060432


In [252]:
gain_root = table_gains.index[0]
gain_root

'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--'

In [253]:
left_gain_df = df[df[gain_root] == 1]
right_gain_df = df[df[gain_root] == 0]

In [254]:
right_gain_table = gain(right_gain_df)
right_gain_table

,n(tL),n(tR),"n(tL,C)","n(tL,NC)","n(tR,C)","n(tR,NC)",PL,PR,"H(s,t)",H(t),gain(s)
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,14,208,14,0,67,141,0.063063,0.936937,0.849490,0.946647,0.097158
PPP2R1A_GRCh37_19:52715971-52715971_Missense-Mutation_SNP_C-C-G_C-C-T,24,198,0,24,81,117,0.108108,0.891892,0.870505,0.946647,0.076143
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,15,207,14,1,67,140,0.067568,0.932432,0.870831,0.946647,0.075816
HAS2_GRCh37_8:122625937-122625937_3'UTR_DEL_A-A--,8,214,8,0,73,141,0.036036,0.963964,0.892523,0.946647,0.054125
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,8,214,8,0,73,141,0.036036,0.963964,0.892523,0.946647,0.054125
...,...,...,...,...,...,...,...,...,...,...,...
SDK2_GRCh37_17:71335062-71335062_Silent_SNP_G-G-A,0,222,0,0,81,141,0.000000,1.000000,0.946647,0.946647,0.000000
DNAH2_GRCh37_17:7696521-7696521_Nonsense-Mutation_SNP_C-C-T,0,222,0,0,81,141,0.000000,1.000000,0.946647,0.946647,0.000000
DHRS7C_GRCh37_17:9674865-9674865_Missense-Mutation_SNP_G-G-T,0,222,0,0,81,141,0.000000,1.000000,0.946647,0.946647,0.000000
CEP76_GRCh37_18:12701088-12701088_Nonsense-Mutation_SNP_C-C-A,0,222,0,0,81,141,0.000000,1.000000,0.946647,0.946647,0.000000


In [255]:
left_gain_table = gain(left_gain_df)
left_gain_table

,n(tL),n(tR),"n(tL,C)","n(tL,NC)","n(tR,C)","n(tR,NC)",PL,PR,"H(s,t)",H(t),gain(s)
HGSNAT_GRCh37_8:43002099-43002099_Frame-Shift-Del_DEL_A-A--,1,27,0,1,27,0,0.035714,0.964286,0.000000,0.222285,0.222285
CAPS2_GRCh37_12:75669988-75669988_3'UTR_DEL_T-T--,1,27,0,1,27,0,0.035714,0.964286,0.000000,0.222285,0.222285
FAT3_GRCh37_11:92616097-92616097_Missense-Mutation_SNP_G-G-A,1,27,0,1,27,0,0.035714,0.964286,0.000000,0.222285,0.222285
TNS3_GRCh37_7:47474967-47474967_Silent_SNP_C-C-T,1,27,0,1,27,0,0.035714,0.964286,0.000000,0.222285,0.222285
TTC14_GRCh37_3:180327408-180327408_Intron_DEL_T-T--,1,27,0,1,27,0,0.035714,0.964286,0.000000,0.222285,0.222285
...,...,...,...,...,...,...,...,...,...,...,...
MIR1283-1_GRCh37_19:54191812-54191812_RNA_SNP_G-G-A,0,28,0,0,27,1,0.000000,1.000000,0.222285,0.222285,0.000000
FPR3_GRCh37_19:52327411-52327411_Missense-Mutation_SNP_G-G-A,0,28,0,0,27,1,0.000000,1.000000,0.222285,0.222285,0.000000
NAPSA_GRCh37_19:50865026-50865026_Missense-Mutation_SNP_C-C-T,0,28,0,0,27,1,0.000000,1.000000,0.222285,0.222285,0.000000
MYH14_GRCh37_19:50758526-50758526_Silent_SNP_G-G-A,0,28,0,0,27,1,0.000000,1.000000,0.222285,0.222285,0.000000


In [256]:
def get_gain_tree(df):
    root_node = gain(df).index[0]
    left_child = df[df[root_node]==1]
    right_child = df[df[root_node]==0]
    left_node = gain(left_child).index[0]
    right_node = gain(right_child).index[0]
    return root_node, left_child, right_child, left_node, right_node

In [257]:
root_node_gain, left_child_gain, right_child_gain, left_node_gain, right_node_gain = get_gain_tree(df)

In [270]:
dot = Digraph()

dot.node('A', root_node_gain[:5])
dot.node('B', left_node_gain[:5])
dot.node('C', right_node_gain[:5])
dot.node('D', 'A1')
dot.node('E', 'A2')
dot.node('F', 'B1')
dot.node('G', 'B2')

dot.edge('A', 'B')
dot.edge('A', 'C')
dot.edge('B', 'D')
dot.edge('B', 'E')
dot.edge('C', 'F')
dot.edge('C', 'G')

dot.view()

'Digraph.gv.pdf'

In [259]:
def get_gain_conditions(df):
    root_node, left_child, right_child, left_node, right_node = get_gain_tree(df)
    print(f"The root node of the root is {root_node} \nThe left node of the root is {left_node} \nThe right node of the root is {right_node}")
    leaf_A1 = left_child[left_child[left_node]==1]
    leaf_A2 = left_child[left_child[left_node]==0]
    leaf_B1= right_child[right_child[right_node]==1]
    leaf_B2 = right_child[right_child[right_node]==0]
    
    # conditions
    A1 = leaf_A1['class'].str.startswith('C').astype(int).sum() > leaf_A1['class'].str.startswith('NC').astype(int).sum()
    A2 = leaf_A2['class'].str.startswith('C').astype(int).sum() > leaf_A2['class'].str.startswith('NC').astype(int).sum()
    B1 = leaf_B1['class'].str.startswith('C').astype(int).sum() > leaf_B1['class'].str.startswith('NC').astype(int).sum()
    B2 = leaf_B2['class'].str.startswith('C').astype(int).sum() > leaf_B2['class'].str.startswith('NC').astype(int).sum()
    
    conditions = [A1, A2, B1, B2]
    
    return conditions

In [260]:
def gain_classification(df, sample, tree_nodes):
    conditions = get_gain_conditions(df)
    S = df[df['class']==sample]
    root_node, left_node, right_node = tree_nodes
    A1, A2, B1, B2 = conditions
    if S[root_node].iloc[0] == 1:
        if S[left_node].iloc[0]==1:
            if A1: #false
                return "C Sample"
            else:
                return "NC Sample"
        else:
            if A2: #true
                return "C Sample"
            else:
                return "NC Sample"
    else:
        if S[right_node].iloc[0]==1:
            if B1: #true
                return "C Sample"
            else:
                return "NC Sample"
        else:
            if B2: #false
                return "C Sample"
            else:
                return "NC Sample"

In [261]:
def build_gain_tree_and_conditions(df):
    root_node = gain(df).index[0]
    left_child = df[df[root_node]==1]
    right_child = df[df[root_node]==0]
    left_node = gain(left_child).index[0]
    right_node = gain(right_child).index[0]
    
    leaf_A1 = left_child[left_child[left_node]==1]
    leaf_A2 = left_child[left_child[left_node]==0]
    leaf_B1= right_child[right_child[right_node]==1]
    leaf_B2 = right_child[right_child[right_node]==0]
    
    # A1 = leaf_A1['class'].str.startswith('C').astype(int).sum() > leaf_A1['class'].str.startswith('NC').astype(int).sum()
    # A2 = leaf_A2['class'].str.startswith('C').astype(int).sum() > leaf_A2['class'].str.startswith('NC').astype(int).sum()
    # B1 = leaf_B1['class'].str.startswith('C').astype(int).sum() > leaf_B1['class'].str.startswith('NC').astype(int).sum()
    # B2 = leaf_B2['class'].str.startswith('C').astype(int).sum() > leaf_B2['class'].str.startswith('NC').astype(int).sum()
    class_array = leaf_A1['class'].values.astype(str)
    A1 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    class_array = leaf_A2['class'].values.astype(str)
    A2 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    class_array = leaf_B1['class'].values.astype(str)
    B1 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    class_array = leaf_B2['class'].values.astype(str)
    B2 = np.char.startswith(class_array, 'C').sum() > np.char.startswith(class_array, 'NC').sum()
    
    return root_node, left_node, right_node, A1, A2, B1, B2
    

In [262]:
def gain_train_and_test(train, test, group):
    print(f"Initializing Gain Training and Testing for Group{group} \n")
    root_node, left_node, right_node, A1, A2, B1, B2 = build_gain_tree_and_conditions(train)
    # print(f"{A1},{A2},{B1},{B2}")
    print(f"Decision Tree: {root_node[:5]}, {left_node[:5]}, {right_node[:5]}")
    classification_results = Classify_test(test, root_node, left_node, right_node, A1, A2, B1, B2)
    pred_actual_df = predicted_actual(classification_results)
    Confusion_Matrix_values = ConfusionMatrix(pred_actual_df)
    print("Confusion_Matrix:", Confusion_Matrix_values, '\n')
    metrics_final = ClassificationMetrics(Confusion_Matrix_values)
    print(metrics_final)
    return metrics_final

In [263]:
gain_train_A, gain_test_A = pd.concat([split_1, split_2], axis=0), split_3
gain_metrics_A = gain_train_and_test(gain_train_A, gain_test_A, 'A')

Initializing Gain Training and Testing for GroupA 

Decision Tree: RPL22, GOT1_, DOCK3
Confusion_Matrix: [17, 44, 1, 22] 

{'Accuracy': 72.619, 'Sensitivity': 43.59, 'Specificity': 97.778, 'Precision': 94.444, 'Miss_rate': 56.41, 'False_Discovery_Rate': 5.556, 'False_Omission_Rate': 33.333}


In [264]:
gain_metrics_B = gain_train_and_test(train_B, test_B, 'B')

Initializing Gain Training and Testing for GroupB 

Decision Tree: RPL22, PTEN_, DOCK3
Confusion_Matrix: [13, 48, 0, 21] 

{'Accuracy': 74.39, 'Sensitivity': 38.235, 'Specificity': 100.0, 'Precision': 100.0, 'Miss_rate': 61.765, 'False_Discovery_Rate': 0.0, 'False_Omission_Rate': 30.435}


In [265]:
gain_metrics_C = gain_train_and_test(train_C, test_C, 'B')

Initializing Gain Training and Testing for GroupB 

Decision Tree: RNF43, GOT1_, RPL22
Confusion_Matrix: [8, 48, 1, 27] 

{'Accuracy': 66.667, 'Sensitivity': 22.857, 'Specificity': 97.959, 'Precision': 88.889, 'Miss_rate': 77.143, 'False_Discovery_Rate': 11.111, 'False_Omission_Rate': 36.0}


In [268]:
def average_metrics(metrics_A, metrics_B, metrics_C):
    avg_met = {}
    for key in metrics_A:
        avg_met[key] = str(((metrics_A[key] + metrics_B[key] + metrics_C[key])/3)) + '%'
    
    print(avg_met)

In [269]:
average_metrics(gain_metrics_A, gain_metrics_B, gain_metrics_C)

{'Accuracy': '71.22533333333334%', 'Sensitivity': '34.894%', 'Specificity': '98.57900000000001%', 'Precision': '94.44433333333335%', 'Miss_rate': '65.106%', 'False_Discovery_Rate': '5.555666666666667%', 'False_Omission_Rate': '33.256%'}
